In [1]:
import torch
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, PreTrainedModel, PretrainedConfig, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
from einops import rearrange
from typing import Dict
from termcolor import cprint
import argparse
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from pyserini.search.lucene import LuceneSearcher




class Retriever:
    def __init__(self, args):

        self.query_model_path = args.dense_retriever_path
        self.retriever_model = args.retriever_model

        # Forced change
        # self.retriever_model = args.model
        self.device = args.device
        if self.query_model_path is None or args.retriever_model == "sparse":
            self.query_model, self.key_model, self.tokenizer = None, None, None
            self.max_seq_len = 512
        elif self.retriever_model == "colbert":
            print("self.query_model_path", self.query_model_path)
            _config = PeftConfig.from_pretrained(self.query_model_path)
            print("_config",_config)
            base_model = _config.base_model_name_or_path
            print("base_model", base_model)
            self.tokenizer = AutoTokenizer.from_pretrained(args.search_tokenizer)

            self.query_model = ColBERT.from_pretrained(base_model)
            self.query_model = PeftModel.from_pretrained(self.query_model, self.query_model_path)
            self.query_model.eval()
            self.query_model.to(self.device)
            self.max_seq_len = min(self.tokenizer.model_max_length, 512)
        elif self.retriever_model=="bge":
            self.tokenizer = AutoTokenizer.from_pretrained(args.search_tokenizer)
            self.query_model = AutoModelForSequenceClassification.from_pretrained(self.query_model_path)
            self.query_model.eval()
            self.query_model.to(self.device)
        else:
            raise NotImplementedError


    def retrieve(self, q, documents, n=1, n_cands=-1, return_rank=False):
        if self.query_model is None: return documents[0]

        if self.retriever_model == "bge":
            pairs = [[q, doc] for doc in documents]
            with torch.no_grad():
                inputs = self.tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(self.device)
                scores = self.query_model(**inputs, return_dict=True).logits.view(-1, ).float()
            ranking = sorted([(idx, score) for idx, score in enumerate(scores)], key=lambda x: x[1], reverse=True)
            documents = sorted([(doc, score) for doc, score in zip(documents, scores)], key=lambda x: x[1], reverse=True)
            documents = [doc[0] for doc in documents]
            
        elif self.retriever_model in ["colbert"]:
            query_outputs = self.tokenizer(q, return_tensors='pt', max_length=self.max_seq_len, padding='max_length', truncation=True)
            key_outputs = self.tokenizer(documents, return_tensors='pt', max_length=self.max_seq_len, padding='max_length', truncation=True)

            query_inputs = {k:v.to(self.device) for k, v in query_outputs.items()}
            key_inputs = {k:v.to(self.device) for k, v in key_outputs.items()}
            # pairs = [[q,doc]for doc in documents]
            # inputs = self.tokenizer (pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
            scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
            with torch.no_grad():
                similarity = self.query_model(**inputs, return_dict=True).logits.view(-1, ).float()
            scores = similarity.flatten().tolist()
            ranking = sorted([(idx, score) for idx, score in enumerate(scores)], key=lambda x: x[1], reverse=True)
            documents = sorted([(doc, score) for doc, score in zip(documents, scores)], key=lambda x: x[1], reverse=True)
            documents = [doc[0] for doc in documents]

        else:
            raise NotImplementedError

        if return_rank: return documents, ranking
        return documents


/home/yuj49/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class ColBERTConfig(PretrainedConfig):
    compression_dim: int = 768
    dropout: float = 0.0
    return_vecs: bool = False
    trainable: bool = True

class ColBERT(PreTrainedModel):
    """
    ColBERT model from: https://arxiv.org/pdf/2004.12832.pdf
    We use a dot-product instead of cosine per term (slightly better)
    """
    config_class = ColBERTConfig
    base_model_prefix = "bert_model"

    def __init__(self, cfg, n_cands=8) -> None:
        super().__init__(cfg)
        
        self.bert = AutoModel.from_pretrained(cfg._name_or_path)

        # for p in self.bert.parameters():
        #     p.requires_grad = cfg.trainable

        self.compressor = torch.nn.Linear(self.bert.config.hidden_size, cfg.compression_dim)

        self.n_cands = n_cands
        print(f"Model n_cands: {self.n_cands}")

    def forward(self,
                query: Dict[str, torch.LongTensor],
                document: Dict[str, torch.LongTensor]):

        query_vecs = self.forward_representation(query)
        document_vecs = self.forward_representation(document)

        score = self.forward_aggregation(query_vecs, document_vecs, query["attention_mask"], document["attention_mask"])
        return score

    def forward_representation(self,
                               tokens,
                               sequence_type=None) -> torch.Tensor:
        
        vecs = self.bert(**tokens)[0] # assuming a distilbert model here
        vecs = self.compressor(vecs)

        # # if encoding only, zero-out the mask values so we can compress storage
        # if sequence_type == "doc_encode" or sequence_type == "query_encode": 
        #     vecs = vecs * tokens["tokens"]["mask"].unsqueeze(-1)

        return vecs

    def forward_aggregation(self, query_vecs, document_vecs, query_mask, document_mask):
        # query_vecs: B x N x D
        # doc_vecs: (B * k) x N x D

        # Unsqueeze query vector
        _bsz = query_vecs.shape[0]
        n_cands = document_vecs.shape[0] // _bsz
        query_vecs_dup = query_vecs.repeat_interleave(n_cands, dim=0).contiguous()

        score = torch.bmm(query_vecs_dup, document_vecs.transpose(1, 2))
        exp_mask = document_mask.bool().unsqueeze(1).expand(-1, score.shape[1], -1)
        score[~exp_mask] = - 10000

        # max pooling over document dimension
        score = score.max(-1).values
        query_mask_dup = query_mask.repeat_interleave(n_cands, dim=0).contiguous()

        score[~(query_mask_dup.bool())] = 0
        score = rearrange(score.sum(-1), '(b n) -> b n', n=n_cands) # B x k 
        return score


In [4]:

def inference(args):
    
    tokenizer = AutoTokenizer.from_pretrained(args.generate_tokenizer)
    model = AutoModelForSeq2SeqLM.from_pretrained(args.generate_model)
    model.to(args.device)
    model.eval()
    if args.knowledge_base=="wiki":
        knowledge_name="enwiki-paragraphs"
    elif args.knowledge_base=="pubmed":
        knowledge_name="beir-v1.0.0-bioasq-flat"
    

    with open(args.input_file_path, 'r') as f, open(args.output_file_path, "w") as output_file:
        if args.type=="first_inference":
            data = []
            for i in f.readlines():
                read_file = json.loads(i)
                # data.append(read_file["article"])  
                input_ids = tokenizer.encode(read_file["article"], return_tensors="pt")
                input_ids = input_ids.to(args.device) 
           
                with torch.no_grad():  
                    generated_ids = model.generate(
                        input_ids,
                        max_length=args.max_input_length + args.max_output_length,
                        num_beams=5,
                        no_repeat_ngram_size=2,
                        early_stopping=True
                    )
    
                generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                print("Generated text:", generated_text)
    
                output_file.write(generated_text + "\n") 
                
        elif args.type=="rag_inference":
            data = []
            for i in f.readlines():
                data.append(i) 
            dense_retriever = Retriever(args)

            for line in tqdm(data):
            
                search_q = line.split("\n")[0]
                if len(search_q.split(" "))>1024:
                    search_q = search_q[:1024]
                
                searcher = LuceneSearcher.from_prebuilt_index(knowledge_name)
                docs = []
                for hit in searcher.search(search_q,k=args.mix_knowledge_num):
                    res = searcher.doc(hit.docid).raw()
                    docs.append(res)
                documents = dense_retriever.retrieve(search_q, docs, n=args.n_docs, n_cands=args.n_cands)
                knowledge = documents[:args.n_docs]
                new_input = search_q + " ".join(knowledge)
                # need to know the max input search size
                if len(new_input) > args.max_search_length:
                    new_input = new_input[:args.max_search_length]
    
                input_ids = tokenizer.encode(new_input, return_tensors="pt")
                input_ids = input_ids.to(args.device) 
    
           
                with torch.no_grad():  
                    generated_ids = model.generate(
                        input_ids,
                        max_length=args.max_input_length + args.max_output_length,
                        num_beams=5,
                        no_repeat_ngram_size=2,
                        early_stopping=True
                    )
    
                generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
                print("Generated text:", generated_text)
    
                output_file.write(generated_text + "\n")


In [7]:
class Args:
    def __init__(self):
        self.knowledge_base = "wiki"
        self.retriever_model = "bge"
        self.dense_retriever_path = "save/bge_models_plos" # save/colbert_lr1e-3/model/
        self.search_tokenizer = "save/bge_models_plos"
        
        self.generate_tokenizer = "allenai/led-base-16384"
        self.generate_model = "laysumm_PLOS_cpt"
        
        self.device = "cuda:2"
        self.n_docs = 5
        self.n_cands = 8
        self.max_input_length = 16384
        self.max_search_length = 512
        self.max_output_length = 512
        self.output_file_path = "output/plos_second_bge.txt"
        self.input_file_path = "output/plos.txt"
        self.type = "rag_inference" #  first_inference
        self.mix_knowledge_num = 5

args = Args()


In [8]:
# dense_retriever = Retriever(args)  # 假设你的Retriever类可以接受一个Args实例或字典
torch.cuda.set_device(args.device)  # 设置使用的设备

# 运行推理
inference(args)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 14%|█▏      | 1/7 [00:03<00:19,  3.28s/it]

Generated text: Gene expression varies widely between individuals of a population, and regulatory change can underlie phenotypes of evolutionary and biomedical interest. A key question in the field is how DNA sequence variants impact gene expression, with most mechanistic studies to date focused on the effects of genetic change on regulatory regions upstream of protein-coding sequence. By contrast, the role of RNA 3′-end processing in regulatory variation remains largely unknown, owing in part to the challenge of identifyi


 29%|██▎     | 2/7 [00:05<00:12,  2.41s/it]

Generated text: The live attenuated simian immunodeficiency virus ( LASIV ) vaccine SIVΔnef is one of the most effective vaccines in inducing protection against wild-type lentiviral challenge, yet little is known about the mechanisms underlying its remarkable protective efficacy. Here, we use deep sequencing technology to deconstruct the CD8 T cell response, to identify the regions of immune pressure and viral escape, and to delineate the effect of epitope escape on the evolution of CD 8 T cells. We demonstrate that the inactivation of T lymphocytes is a major factor in the development of immunity to the virus.


 43%|███▍    | 3/7 [00:07<00:09,  2.39s/it]

...

Generated text: Parkinson’s disease ( PD ) is the most common neurodegenerative movement disorder. GBA mutations in the glucocerebrosidase ( GCase ) gene result in Gaucher disease, a lysosomal storage disease characterized by neuronal loss, astrogliosis, microgliosis and perivascular accumulation of neurons. Recent genetic studies have revealed that GMA mutations confer a strong risk for sporadic Parkinson's disease in medaka. To investigate the mechanisms that lead to PD, we generated a new neuronopathic GD model that is 


 86%|██████▊ | 6/7 [00:12<00:01,  1.95s/it]

Generated text: Listeria monocytogenes is a facultative intracellular pathogen capable of inducing a robust cell-mediated immune response to sub-lethal infection. The capacity of L. monoclonal pathogens to escape from the phagosome and enter the host cell cytosol is paramount for the induction of long-lived CD8 T cell–mediated protective immunity. Here, we show that the immune system discriminates between pathogens that reside in distinct subcellular locations, and via a MyD88- and IL-10-dependent mechanism, impacts the po


100%|████████| 7/7 [00:15<00:00,  2.21s/it]

Generated text: Primary open-angle glaucoma ( POAG ) is a group of diseases that affect approximately 60 million people worldwide and is the leading cause of blindness. The SIX6 gene family consists of six members, all of which contain two shared protein domains; a DNA binding homeobox domain and a domain, which binds downstream effector molecules. Genome-wide association studies have previously identified a significant genetic association ( rs10483727, odds ratio ( OR ) = 1. 32, p.=3. 87×10−11 ). We sequenced the coding of a gene that is involved in the binding of DNA and RNA molecules to downstream proteins. In this study, we found that the gene is associated with a large number of genetic associations.


In [9]:
class Args:
    def __init__(self):
        self.knowledge_base = "wiki"
        self.retriever_model = "bge"
        self.dense_retriever_path = "save/bge_models_elife" # save/colbert_lr1e-3/model/
        self.search_tokenizer = "save/bge_models_elife"
        
        self.generate_tokenizer = "allenai/led-base-16384"
        self.generate_model = "laysumm_eLife_cpt"
        
        self.device = "cuda:2"
        self.n_docs = 5
        self.n_cands = 8
        self.max_input_length = 16384
        self.max_search_length = 512
        self.max_output_length = 512
        self.output_file_path = "output/elife_second_bge.txt"
        self.input_file_path = "output/elife.txt"
        self.type = "rag_inference" #  first_inference
        self.mix_knowledge_num = 5

args = Args()
# dense_retriever = Retriever(args)  # 假设你的Retriever类可以接受一个Args实例或字典
torch.cuda.set_device(args.device)  # 设置使用的设备

# 运行推理
inference(args)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|      | 1/241 [00:02<10:07,  2.53s/it]

Generated text: Cell-fate decisions are controlled by a process called reprograming, in which the cell’s chromatin is switched on or off. This process is controlled, on the one hand, by signaling from a cell niche to the recipient cell and then switching it off or on again. However, it is not clear how this process works in intact tissues. To investigate this question, Tursun, Patel et al. studied the germline of a worm called C. elegans, and found that a gene for a protein called GLP-1/Notch is turned on and off in this worm. The experiments revealed that this gene was switched off and on in the worm, but it was unclear how it could be controlled in an intact tissue.


  1%|      | 2/241 [00:04<09:05,  2.28s/it]

Generated text: Klebsiella pneumoniae is a bacterium that can cause pneumonia, sepsis and other life-threatening infections in the lungs of people with pneumonia. The disease is caused by a type of immune system called the neutrophil, which is able to bind to the bacteria that cause the infection and fight against them to prevent them from spreading to other patients with the disease. This is known as the “neutrophils” because they are the first responder cell type for fighting against invading bacteria and are also vulnerable to infection by other bacteria called resista


  1%|      | 3/241 [00:06<08:11,  2.06s/it]

...

Generated text: RNA molecules are made up of molecules called messenger RNAs ( or RNA molecules for short ) that make up proteins and other molecules of RNA. The RNA molecule that makes up a protein is produced by a process called transcription, which is carried out by the enzyme that is responsible for making the messenger molecule. However, it is not clear how the transcription process works without the help of proteins. A group of genes called ribonucleotide derivatives are thought to have evolved from the RNA World, but it was not known how they did so. Now, Nelson, Reiss et al. have discovered that the process of making a new protein using a gene called the riboswitch gene has evolved in a similar way to that of the DNA World gene. This gene is made of a molecule called RNA that contains a number of other genes that are involved in the production of new proteins, but they are not well understood how these genes evolved to make the protein they make. These genes have been linked to

100%|███▉| 240/241 [10:52<00:02,  2.97s/it]

Generated text: Neurons communicate with each other via a network of synapses. Each synapse contains a single vesicle, which is released into the synaptic cleft by a protein called synapsin. This protein is responsible for controlling the number of synaptic connections between neurons in the brain and other parts of the body. Synapses are made up of many different types of neurons and each type of neuron is able to communicate in a coordinated manner with one another by releasing a small molecule called a neurotransmitter at a time. However, it was previously thought that only one neuron can release a certain amount of neurotransmitters at the same time—that is to say that each neuron has a specific amount to release at any given time.


100%|████| 241/241 [10:54<00:00,  2.72s/it]

Generated text: HIV-1 is an infectious virus that infects humans and other animals. The virus is made up of a protein called Gag, which is found on the surface of the host’s plasma membrane. This protein is able to package the genome of an infected virus in a way that makes it easier for it to assemble into a virus. However, it is not clear how this process is carried out in the laboratory. To investigate this question, Kuzembayeva et al. used a minimal system consisting of purified RNAs called giant unilamellar vesicles ( GUVs ) to create a small system that could be used to make the virus more compact and easier to build a new virus from.


In [12]:

# simple save
class Args:
    def __init__(self):
        self.knowledge_base = "wiki"
        # self.retriever_model = "bge"
        # self.dense_retriever_path = "save/bge_models" # save/colbert_lr1e-3/model/
        # self.search_tokenizer = "save/bge_models"
        
        self.generate_tokenizer = "allenai/led-base-16384"
        self.generate_model = "laysumm_PLOS_cpt"
        
        self.device = "cuda:2"
        self.n_docs = 5
        self.n_cands = 8
        self.max_input_length = 16384
        self.max_search_length = 512
        self.max_output_length = 512
        self.output_file_path = "output/plos.txt"
        self.input_file_path = "data/PLOS_val.jsonl"
        self.type = "first_inference" #  first_inference
        self.mix_knowledge_num = 5

args = Args()
# dense_retriever = Retriever(args)  # 假设你的Retriever类可以接受一个Args实例或字典
torch.cuda.set_device(args.device)  # 设置使用的设备

# 运行推理
inference(args)


Generated text: Gene expression varies widely between individuals of a population, and regulatory change can underlie phenotypes of evolutionary and biomedical interest. A key question in the field is how DNA sequence variants impact gene expression, with most mechanistic studies to date focused on the effects of genetic change on regulatory regions upstream of protein-coding sequence. By contrast, the role of RNA 3′-end processing in regulatory variation remains largely unknown, owing in part to the challenge of identifying functional elements in untranslated regions. In this work, we conducted a genomic survey of transcript ends in lymphoblastoid cells from genetically distinct human individuals. Our analysis mapped the cis-regulatory architecture of transcripts in these cells, finding that transcript end positions did not fall randomly, but rather preferentially flanked the locations of regulatory elements, including miRNA sites. The usage of these transcript length forms and motifs

Token indices sequence length is longer than the specified maximum sequence length for this model (16950 > 16384). Running this sequence through the model will result in indexing errors


Generated text: Primary open-angle glaucoma ( POAG ) is a group of diseases that affect approximately 60 million people worldwide and is the leading cause of blindness. The SIX6 gene family consists of six members, all of which contain two shared protein domains; a DNA binding homeobox domain and a domain, which binds downstream effector molecules. Genome-wide association studies have previously identified a significant genetic association ( rs10483727, odds ratio ( OR ) = 1. 32, p.=3. 87×10−11 ). We sequenced the coding and regulatory regions in a case-control dataset; we found both common and rare coding variants, including five rare and one common variant, Asn141His. Five variants were found in PoAG cases, while the sixth variant was benign. Our results, in combination with previous studies, lead us to hypothesize that the loss of retinal ganglion cells that an individual possesses increases the risk of developing the disease.


../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [128,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [12

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
